In [107]:
import numpy as np
import pandas as pd 
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
from typing import Dict
import logging
# logging.getLogger().setLevel(logging.INFO)

import gurobipy as gp
from gurobipy import GRB
from itertools import product

In [108]:
filepath = '../data/US_latest_data_20220428.parquet' 
return_filepath = '../data/Exp_Return_Mom.parquet'
df = pd.read_parquet(filepath, engine = 'pyarrow')
expected_return = pd.read_parquet(return_filepath, engine = 'pyarrow')

In [109]:
def clean_stock_data(df: pd.DataFrame) -> pd.DataFrame:
    # select required months
    df = df[["DateYYYYMM","ws_id","Price","Div","mom","mktcap","median_volume_usd"]]
    df = df.rename(columns={"DateYYYYMM": "date", "ws_id":"asset", "Price":"price", "Div":"div",
           "mom":"factor"})
    
    # Correct datetime 
    df['date'] = pd.to_datetime(df['date'], format = "%Y%m")
    
    # Change units for dollar valued data
    df['mktcap'] = df['mktcap'] *1000000
    df['median_volume_usd'] = df['median_volume_usd'] *1000000
    
    # replace missing values
    df['div'] = df['div'].fillna(0)
    df['price'] = df['price'].dropna()
    
    return df

def clean_return_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={"Date": "date", "Asset":"asset", "Exp_Return":"exp_return"})
    df['date'] = pd.to_datetime(df['date'], format = "%Y%m")
    
    return df

In [110]:
df = clean_stock_data(df)
expected_return = clean_return_data(expected_return)

In [113]:
def filter_eligible_investments(df: pd.DataFrame, date: pd.Timestamp, ratio: float = 0.25, size: int = 1000) -> list:
    """
    Return the dataframe containing the top ratio% assets based on the factor of the top 1000 assets  of top asset based on market cap.
    Remove assets that no longer exist 1 period ahead
    """
    factor_num = int(ratio*size)
    
    df_current = df[df['date'] == date]
    df_next = df[df['date'] == date + pd.DateOffset(months=1)]
    
    asset_diff = set(df_current['asset']) - set(df_next['asset'])
    df_current = df_current[~df_current['asset'].isin(asset_diff)]
    
    df_by_mktcap = df_current.sort_values(by = 'mktcap', ascending=False).reset_index(drop=True)[:size]
    df_by_factor = df_by_mktcap.sort_values(by = 'factor', ascending=False).reset_index(drop=True)[:factor_num] 
     
    return df_by_factor.asset.to_list()

def update_assets(df: pd.DataFrame, valuation: float) -> Dict:
    """
    Return the dictionary of asset_names as the key and price/quantity tuple as the value. The proportion how much to purchase is determined by the  
    """
    
    df['ratio'] = df['mktcap']/df['mktcap'].sum()
    df['quantity'] = np.floor(df['ratio']*valuation/df['price'])
    assets = dict(zip(df['asset'], zip(df['quantity'], df['price'])))
    return assets
    

In [6]:
target_portfolio = {
    pd.to_datetime("197112", format = "%Y%m"): {
        'valuation': 10e8, # CASH+SHARES*PRICE + DIV
        'cash': 10e8,
        'div_paid': 0, #keep for debugging
        # 'index': 1, #Will be added back in if we consider cash injections
        'asset': {}, # {asset: (quantity,price)},
        'turnover': 0,
        'total_cost_explicit': 0,
        'total_cost_implicit': 0,
        # 'cash_flow': 0
    },
}


for date in pd.to_datetime(df['date'].sort_values().unique())[:15]:
    # filter the dataframe to the current date 
    df_current = df[df['date'] == date]
    
    # initialise current date key in portfolio
    target_portfolio[date] = {}
    
    # update dividents paid
    prev_assets = target_portfolio[date-pd.DateOffset(months=1)]['asset'] 
    
    div_paid = 0
    asset_value = 0
    for asset_name, quantity_price in prev_assets.items():
        if asset_name not in df_current.asset.to_list():
            asset_value += quantity_price[0]*quantity_price[1]
        else:
            asset_value += float(quantity_price[0] * df_current.loc[df_current['asset'] == asset_name, 'price'])
            div_paid += float(quantity_price[0]*df_current.loc[df_current['asset'] == asset_name, 'div'])

    
    target_portfolio[date]['div_paid'] = div_paid
    target_portfolio[date]['valuation'] = asset_value + div_paid + target_portfolio[date - pd.DateOffset(months=1)]['cash'] 
    
    # print(f"{date}: {target_portfolio[date]['valuation']}")
    
    # subset to eligible assets names
    asset_list = filter_eligible_investments(df, date)
    df_current = df[(df['date'] == date) | (df['asset'].isin(asset_list))]
    
    # update the assets to the preferred ones
    target_portfolio[date]['asset'] = update_assets(df_current, target_portfolio[date]['valuation'])
    
    # update cash amount
    target_portfolio[date]['cash'] = target_portfolio[date]['valuation'] - sum([quantity_price[0]*quantity_price[1] for asset_name, quantity_price in target_portfolio[date]['asset'].items()])

    

In [7]:
target_portfolio[pd.to_datetime("197208", format = "%Y%m")]['asset']["b'51086'"][0]#['valuation'] # valutation at dec 2020 should be 70037236032 a return of 14.28% calculate by (val_end/val_1)^(12/n)

109.0

## Optimisation 

#### Data:
* $A$: set of assets = (the set of assets we are aiming to trade) $\cup$ (the set of assets we currently own)


* $m_j$ = expected return for asset $j\in A$ 
* $u_j$ = implicit cost lower bound for trading large proportions of asset $j\in A$   
* $v_j$ = implicit cost lower bound for trading small proportions of asset $j\in A$   
* $s$ = explicit cost lower bound for trading large proportions of asset $j\in A$   
* $t$ = explicit cost lower bound for trading small proportions of asset $j\in A$   
* $a_j$ = pre-trade proportion of the portfolio made up by asset $j\in A$ (this value is zero if we do not currently own it)
* $b_j$ = 1 if the asset $j\in A$ is in the top 250 momentum assets and 0 otherwise.
* $e$ = small value 
* $M$ = large value


#### Variables:
* $w_j$: weight of trade for stock $j\in A$
* $c_{Ej}$: explicit cost of trading asset $j\in A$ 
* $c_{Ij}$: implicit cost of trading asset $j\in A$ 
* $x_j$: binary variable, taking value $1$ if the weight of trade for stock $j\in A$ is $0$ and $1$ otherwise.

#### Model:

Max $z = \sum_{j\in A} m_j w_j - c_{Ej} - c_{Ij}$

subject to 
* $c_{Ej} \geq u_j w_j^2, \quad \forall j\in A$ 
* $c_{Ej} \geq v_j w_j, \quad \forall j\in A$ 
* $c_{Ej} \geq -v_j w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq s w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq -s w_j, \quad \forall j\in A$ 
* $c_{Ij} \geq t x_j, \quad \forall j\in A$ 
* $w_j \geq e - M(1-x_j), \quad \forall j \in A$
* $w_j \leq -e + M x_j, \quad \forall j \in A$
* $\sum_{j\in A} w_j = 0$
* $w_j \geq a_j, \quad \forall j \in A$
* $w_j \leq b_j, \quad \forall j \in A$
* $w_j \in [-1,1], x_j \in \{0,1\}, c_{Ij}, c_{Ej} \geq 0, \quad \forall j \in A$
 

In [136]:
def calculate_large_implicit_cost(monthly_portfolio: Dict, df_monthly: pd.DataFrame) -> Dict:
    """
    Calculate the implicit cost of transacting each asset. This formulation of the implicit cost follows the formula
    6*valuation/market_cap
    This is the implicit cost used for larger transactions.
    """
    # print(f"mktcap min:{df_monthly.loc[df_monthly['mktcap'] ==0, 'mktcap']}")
    df_monthly.loc[df_monthly['mktcap'] ==0, 'mktcap'] = np.inf # should the cost be zero if there's no market cap or should it be large?
    valuation = monthly_portfolio['valuation']
    df_monthly.loc[:,'large_implicit_cost'] = 6*valuation/df_monthly['mktcap']   
    
    implicit_cost = dict(zip(df_monthly['asset'], df_monthly['large_implicit_cost']))
    
    return implicit_cost


def calculate_small_implicit_cost(df_monthly: pd.DataFrame) -> Dict:
    """
    Calculate the implicit cost of transacting each asset. This formulation of the implicit cost follows the formula,
    bid_ask_spread/valuation
    This is the implicit cost used for smaller transactions
    """  
    df_monthly.loc[:,'small_implicit_cost'] = 0.0013 * np.sqrt(np.median(df_monthly['mktcap'])/df_monthly['mktcap'])
    implicit_cost = dict(zip(df_monthly['asset'], df_monthly['small_implicit_cost']))
    
    return implicit_cost    


def assemble_monthly_data(df:pd.DataFrame, expected_return:pd.DataFrame, target_portfolio: Dict, date:pd.Timestamp) -> pd.DataFrame:
    
    df_current = df[df.date == date]
    monthly_return = expected_return[expected_return['date'] == date]
    df_current = pd.merge(df_current, monthly_return, on = ['date', 'asset'], how="outer").fillna(0)
    
    buy_list = filter_eligible_investments(df, date)
    df_current.loc[df_current['asset'].isin(buy_list), 'buy'] = 1
    df_current['buy'] = df_current['buy'].fillna(0)
    
    # print(f"df_current: {df_current}")
    
    # if date == pd.to_datetime("197201", format = "%Y%m"):
    #     df_current['sell'] = -1
    # else:
    portfolio_assets = pd.DataFrame.from_dict(target_portfolio[date]['asset'], orient='index')
    portfolio_assets = portfolio_assets.reset_index().rename(columns={'index':'asset'})
    df_current = pd.merge(df_current, portfolio_assets, on = ['asset'], how="outer").fillna(0)
    if date == pd.to_datetime("197201", format = "%Y%m"):
        df_current.loc[df_current['asset'] == 'sell_asset', 'price'] = 1
        # df_current['sell'] = - df_current[0] * df_current[1] / target_portfolio[date]['valuation']
        # print(df_current['sell'])
    # else:
    df_current['sell'] = - df_current[0] * df_current['price'] / target_portfolio[date]['valuation']
    # df_current['sell'] = - df_current[0] * df_current['price'] / target_portfolio[date]['valuation']
    # df_current.to_csv(f"../data/df_current_{date}.csv", index = False) 
    # print(f"portfolio_assets: {df_current}")
    
    forced_sale_list = list(set(target_portfolio[date]['asset']) - set(df.loc[df['date'] == date + pd.DateOffset(months=1), 'asset']))
    df_current.loc[df_current['asset'].isin(forced_sale_list), 'buy'] = df_current.loc[df_current['asset'].isin(forced_sale_list), 'sell']
    
    df_current = df_current[['asset', 'mktcap', 'exp_return', 'price', 'buy', 'sell']]
    
    return df_current

def calculate_optimal_portfolio(df: pd.DataFrame, expected_return: pd.DataFrame, portfolio: Dict, date: pd.Timestamp) -> Dict:
    """
    Find the optimal trade as proportions of the current portfolio valuation. Provide the assets we want to trade along with the total explicit and implicit costs.
    """
    
    # Obtain data    
    df_current = assemble_monthly_data(df, expected_return, portfolio, date)
    print(df_current[df_current['price']==0])
    A = list(df_current['asset'].unique())
    u = calculate_large_implicit_cost(portfolio[date], df_current) 
    v = calculate_small_implicit_cost(df_current)
    s = 0.0001
    t = 5/portfolio[date]['valuation'] 
    mu = dict(zip(df_current['asset'], df_current['exp_return']))
    M = 10e8
    e = 10e-8
    a = dict(zip(df_current['asset'], df_current['sell']))
    b = dict(zip(df_current['asset'], df_current['buy']))
    
    
    # Initialise Model
    m = gp.Model(f'Optimisation_{date}')
    m.Params.LogToConsole = 0
    
    # Define variables
    w = m.addVars(A, vtype = GRB.SEMICONT, lb=-1, ub=1, name='w')
    x = m.addVars(A, vtype = GRB.BINARY, name ='x')
    ce = m.addVars(A, vtype = GRB.CONTINUOUS, name='ce')
    ci = m.addVars(A, vtype = GRB.CONTINUOUS, name='ci')
    
    # Define objective
    m.setObjective(gp.quicksum(mu[j]*w[j] - ce[j] - ci[j] for j in A),GRB.MAXIMIZE)
    
    # Define constraints
    m.addConstrs( ci[j] >= u[j]*w[j]**2 for j in A)
    m.addConstrs( ci[j] >= v[j]*w[j] for j in A)
    m.addConstrs( ci[j] >= -v[j]*w[j] for j in A)
    
    m.addConstrs( ce[j] >= s*w[j] for j in A)
    m.addConstrs( ce[j] >= -s*w[j] for j in A)
    m.addConstrs( ce[j] >= t*x[j] for j in A)
    
    m.addConstrs(w[j] >= e - M*(1-x[j]) for j in A)
    m.addConstrs(w[j] <= -e + M*x[j] for j in A)
    
    m.addConstr(gp.quicksum(w[j] for j in A) == 0)
    m.addConstrs(w[j] >= a[j] for j in A)
    m.addConstrs(w[j] <= b[j] for j in A)
    
    # Solve and collect data
    m.optimize()
    
    turnover = 0
    total_cost_explicit = 0
    total_cost_implicit = 0 
    assets = {}
    
    for v in m.getVars():
        if v.varName.startswith("ce"):
            total_cost_explicit += v.x
        elif v.varName.startswith("ci"):
            total_cost_implicit += v.x
    
    valuation = portfolio[date]['valuation'] - total_cost_explicit - total_cost_implicit # check
    
    for v in m.getVars():
        if v.varName.startswith("w"):
            turnover += abs(v.x)
           
            varname = v.varName
            varname = varname.strip("w[").strip("]")
            price = float(df_current.loc[df_current.asset == varname, "price"])

            if v.x > 0:
                assets[varname] = (np.floor(v.x * valuation/price), price)
            elif (v.x <0) and (portfolio[date]['asset'][varname][0] + v.x * valuation/price >= 0):
                assets[varname] = (np.floor(portfolio[date]['asset'][varname][0] + v.x * valuation/price), price)
                # possible that this will under-state trading costs - why is it that we can sell more than we own? possibly valuation error?
    
    output_dict = {
        "turnover": turnover/2,
        "total_cost_explicit": total_cost_explicit,
        "total_cost_implicit": total_cost_implicit,
        "asset": assets
    }
    
    return output_dict


def optimise_markdowns(target_portfolio: Dict, n_dates: int = len(df['date'].unique())) -> Dict:
    """
    Loop through the set of dates, update the portfolio for each date, find the optimal trades and finally add the optimal values to the dictionary
    """
    for date in pd.to_datetime(df['date'].sort_values().unique())[:10]:
        # filter the dataframe to the current date 
        df_current = df[df['date'] == date]
        
        # initialise current date key in portfolio
        target_portfolio[date] = {}
        
        # update dividents paid
        prev_assets = target_portfolio[date-pd.DateOffset(months=1)]['asset'] 
        
        div_paid = 0
        asset_value = 0
        for asset_name, quantity_price in prev_assets.items():
            if asset_name not in df_current.asset.to_list():
                asset_value += quantity_price[0]*quantity_price[1]
            else:
                asset_value += float(quantity_price[0] * df_current.loc[df_current['asset'] == asset_name, 'price'])
                div_paid += float(quantity_price[0]*df_current.loc[df_current['asset'] == asset_name, 'div'])
        
        target_portfolio[date]['div_paid'] = div_paid
        target_portfolio[date]['valuation'] = asset_value + div_paid + target_portfolio[date - pd.DateOffset(months=1)]['cash'] 
        target_portfolio[date]['asset'] = target_portfolio[date-pd.DateOffset(months=1)]['asset'] 
        
        logging.info(f" Valuation at time {date} is {target_portfolio[date]['valuation']}")
        
        # find optimal asset allocation
        optimised_outputs = calculate_optimal_portfolio(df, expected_return, target_portfolio, date)
        target_portfolio[date]['asset'] = optimised_outputs['asset']
        target_portfolio[date]['turnover'] = optimised_outputs['turnover']
        target_portfolio[date]['total_cost_explicit'] = optimised_outputs['total_cost_explicit']
        target_portfolio[date]['total_cost_implicit'] = optimised_outputs['total_cost_implicit']
        
        # update cash amount
        target_portfolio[date]['cash'] = target_portfolio[date]['valuation'] - sum([quantity_price[0]*quantity_price[1] for asset_name, quantity_price in target_portfolio[date]['asset'].items()])
        
    return target_portfolio
    

In [115]:
df[df['date'] == pd.to_datetime("197201", format = "%Y%m")]

,date,asset,price,div,factor,mktcap,median_volume_usd
235,1972-01-01,b'10006',1.0,0.0,0.121682,289644750.0,243225.00000
725,1972-01-01,b'10014',1.0,0.0,-0.187500,23980000.0,17500.00000
5423,1972-01-01,b'10102',1.0,0.0,-0.172548,258610250.0,219275.00000
7407,1972-01-01,b'10137',1.0,0.0,-0.068361,499848000.0,402675.00000
8648,1972-01-01,b'10145',1.0,0.0,0.186626,885236500.0,307570.36800
...,...,...,...,...,...,...,...
630352,1972-01-01,b'49584',1.0,0.0,-0.132022,9065000.0,12600.03600
633790,1972-01-01,b'49795',1.0,0.0,-0.354546,10125000.0,148837.50000
635301,1972-01-01,b'49963',1.0,0.0,0.053097,19635000.0,86100.00000
738919,1972-01-01,b'58843',1.0,0.0,0.039786,162362750.0,135987.59460


In [139]:
target_portfolio[pd.to_datetime("197204", format = "%Y%m")]

{'div_paid': 273088.06556128006,
 'valuation': 1064033302.76282,
 'asset': {"b'10188'": (94503.0, 1.06504066604457),
  "b'10233'": (675995.0, 1.22295673845965),
  "b'10487'": (134174.0, 0.955621571346),
  "b'10559'": (1990520.0, 0.991170091895681),
  "b'10751'": (186424.0, 1.01129945256408),
  "b'10786'": (1007088.0, 0.972868680924726),
  "b'10874'": (583331.0, 1.18154800147779),
  "b'10890'": (711979.0, 1.10121468331162),
  "b'11105'": (192279.0, 1.13043413940556),
  "b'11308'": (38422642.0, 1.08872575345588),
  "b'11607'": (188129.0, 1.13868724011578),
  "b'11754'": (1422000.0, 1.18440561790201),
  "b'12060'": (18401457.0, 1.09072623473471),
  "b'12095'": (154426.0, 1.26262568759877),
  "b'12749'": (784437.0, 1.14434331550387),
  "b'13119'": (295167.0, 1.04411792052124),
  "b'13936'": (130281.0, 0.930348183568068),
  "b'14090'": (8884488.0, 0.968152388074319),
  "b'15034'": (1492576.0, 0.981651596554876),
  "b'15077'": (123013.0, 1.13690496090545),
  "b'15368'": (5733726.0, 1.1662049

In [138]:
target_portfolio = {
    pd.to_datetime("197112", format = "%Y%m"): {
        'valuation': 0, 
        'cash': 0,
        'div_paid': 0,
        'asset': {'sell_asset':(10e8,1)},
        'turnover': 0,
        'total_cost_explicit': 0,
        'total_cost_implicit': 0,
    },
}

final_portfolio = optimise_markdowns(target_portfolio)

 Valuation at time 1972-01-01 00:00:00 is 1000000000.0
Empty DataFrame
Columns: [asset, mktcap, exp_return, price, buy, sell]
Index: []
 Valuation at time 1972-02-01 00:00:00 is 1045806425.9482231
           asset  mktcap  exp_return  price  buy  sell
1538  sell_asset     0.0         0.0    0.0 -0.0  -0.0
 Valuation at time 1972-03-01 00:00:00 is 1059207696.8133768
Empty DataFrame
Columns: [asset, mktcap, exp_return, price, buy, sell]
Index: []
 Valuation at time 1972-04-01 00:00:00 is 1064033302.76282
Empty DataFrame
Columns: [asset, mktcap, exp_return, price, buy, sell]
Index: []
 Valuation at time 1972-05-01 00:00:00 is 1076711331.8829713
         asset  mktcap  exp_return  price  buy  sell
1530  b'47335'     0.0         0.0    0.0 -0.0  -0.0
 Valuation at time 1972-06-01 00:00:00 is 1074783722.3092735
Empty DataFrame
Columns: [asset, mktcap, exp_return, price, buy, sell]
Index: []
 Valuation at time 1972-07-01 00:00:00 is 1079966545.5421438
Empty DataFrame
Columns: [asset, mktcap, 

In [ ]:
final_portfolio[pd.to_datetime("197210", format = "%Y%m")]

In [ ]:
target_portfolio